In [4]:
import scipy
import random
import portpy.photon as pp
import numpy as np
from cvxpy import *
from scipy import sparse
import sklearn.metrics
import cProfile
import time
import sys
import psutil
from sklearn.decomposition import PCA

In [3]:
data_dir = r'../data'
data = pp.DataExplorer(data_dir=data_dir)
patient_id = 'Lung_Phantom_Patient_1'
data.patient_id = patient_id
ct = pp.CT(data)
structs = pp.Structures(data)
beams = pp.Beams(data)
opt_params = data.load_config_opt_params(protocol_name='Lung_2Gy_30Fx')
structs.create_opt_structures(opt_params)
inf_matrix_sparse = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams)
protocol_name = 'Lung_2Gy_30Fx'
clinical_criteria = pp.ClinicalCriteria(data, protocol_name)
plan_sparse = pp.Plan(ct, structs, beams, inf_matrix_sparse, clinical_criteria)
opt = pp.Optimization(plan_sparse, opt_params=opt_params)
opt.create_cvxpy_problem()

sol_sparse = opt.solve(solver='MOSEK', verbose=True)
dose_sparse_1d = plan_sparse.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions())
x_sparse =sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions()

beams_full = pp.Beams(data, load_inf_matrix_full=True)
inf_matrix_full = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams_full, is_full=True)
plan_full = pp.Plan(ct, structs, beams, inf_matrix_full, clinical_criteria)
dose_full_1d = plan_full.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_full.get_num_of_fractions())

A_full = plan_full.inf_matrix.A
A_sparse = plan_sparse.inf_matrix.A


creating rinds.. This step may take some time due to dilation
rinds created!!
Creating BEV..
Loading sparse influence matrix...
Done
Objective Start
Objective done
Constraints Start
Structure ESOPHAGUS not available!
Structure ESOPHAGUS not available!
Constraints done
                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Oct 27 04:22:33 PM: Your problem has 1946 variables, 14 constraints, and 0 parameters.
(CVXPY) Oct 27 04:22:33 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 27 04:22:33 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 27 04:22:33 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation       

(CVXPY) Oct 27 04:22:43 PM: 21  3.3e-07  1.1e-06  3.1e-11  1.00e+00   4.204653231e+01   4.204653206e+01   2.3e-09  4.34  
(CVXPY) Oct 27 04:22:43 PM: 22  9.5e-08  3.1e-07  4.8e-12  9.56e-01   4.204568395e+01   4.204568387e+01   6.6e-10  4.49  
(CVXPY) Oct 27 04:22:43 PM: 23  7.2e-08  7.8e-08  6.2e-13  1.01e+00   4.204544808e+01   4.204544806e+01   1.7e-10  4.62  
(CVXPY) Oct 27 04:22:43 PM: Optimizer terminated. Time: 4.75    
(CVXPY) Oct 27 04:22:43 PM: 
(CVXPY) Oct 27 04:22:43 PM: 
(CVXPY) Oct 27 04:22:43 PM: Interior-point solution summary
(CVXPY) Oct 27 04:22:43 PM:   Problem status  : PRIMAL_AND_DUAL_FEASIBLE
(CVXPY) Oct 27 04:22:43 PM:   Solution status : OPTIMAL
(CVXPY) Oct 27 04:22:43 PM:   Primal.  obj: 4.2045448085e+01    nrm: 1e+02    Viol.  con: 6e-06    var: 2e-06    cones: 0e+00  
(CVXPY) Oct 27 04:22:43 PM:   Dual.    obj: 4.2045448063e+01    nrm: 6e+03    Viol.  con: 0e+00    var: 8e-07    cones: 0e+00  
------------------------------------------------------------------

In [5]:
# 1. مرکزگرایی داده (اختیاری):
mean = np.mean(A_full, axis=0)
A_full_centered = A_full - mean

# 2. اعمال PCA:
n_components = 542  # تعداد مؤلفه‌های اصلی مورد نظر
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(A_full_centered)

# مؤلفه‌های اصلی به ترتیب مقادیر ویژه بزرگتر به کوچکتر در principal_components ذخیره می‌شوند.


In [9]:
T_Matrix_PCA = principal_components.transpose()

T_Matrix_PCA.shape

(542, 60183)

In [10]:
start_time = time.time()

sol_ls_A_PCA = scipy.sparse.linalg.lsqr(T_Matrix_PCA@A_full, T_Matrix_PCA@dose_full_1d)

end_time = time.time()
total_execution_time = end_time - start_time
print(f"زمان کل اجرای کمترین مربعات ماتریس PCA \n: {total_execution_time} ثانیه")


زمان کل اجرای کمترین مربعات ماتریس PCA 
: 2.0116207599639893 ثانیه


In [12]:
# جواب بهینه
x_star = sol_ls_A_PCA[0]
print(f'x_star :\t {x_star[0:4]}')

#  تعداد مراحل اجرای الگوریتم
number_algorithm_execution_steps = sol_ls_A_PCA[2]
print(f'number_algorithm_execution_steps :\t {number_algorithm_execution_steps}')

#  نرم بردار سمت راست d
Rightـhandـsideـvectorـnormـd = sol_ls_A_PCA[3]
print(f'Rightـhandـsideـvectorـnormـd :\t {Rightـhandـsideـvectorـnormـd}')

#شاخص برای ارزیابی خروجی الگوریتم کمترین مربعات است	هرچقدر این نرم کوچکتر باشد نشانه بهتر بودن جواب بهینه است


#نرم ضرب داخلی ماتریس در جواب
l2_res = sol_ls_A_PCA[4]
print(f'L2 norm of residuals :\t{l2_res}')

#   مجموع مربعات خطای سیستم 
sum_of_square1 = sol_ls_A_PCA[5]
print(f'sum_of_square1 :\t {sum_of_square1}')

#   مجموع مربعات خطای ماتریس معکوس
sum_of_square2 = sol_ls_A_PCA[6]
print(f'sum_of_square2 :\t {sum_of_square2}')

# بردار صفر              ?????????????
Zeroـvector = sol_ls_A_PCA[9]
print(f'Zeroـvector :\t {Zeroـvector[0:24]}')

# کد بازگشتی 
Returnـcode = sol_ls_A_PCA[1]
print(f'Returnـcode :\t {Returnـcode}')

x_star :	 [ 6.53035609e+00  1.87758974e-01 -1.66915830e-04  1.74868622e-03]
number_algorithm_execution_steps :	 161
Rightـhandـsideـvectorـnormـd :	 0.49475562933377715
L2 norm of residuals :	0.49475562933377715
sum_of_square1 :	 1963.8870373593206
sum_of_square2 :	 1194.8402869477388
Zeroـvector :	 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Returnـcode :	 1


In [22]:
nonzero_count_T_Matrix_PCA = np.count_nonzero(T_Matrix_PCA)

print(" PCA تعداد مؤلفه‌های غیر صفر ماتریس \t", nonzero_count_T_Matrix_PCA)


nonzero_count_A_full = np.count_nonzero(A_full)

print("تعداد مؤلفه‌های غیر صفر ماتریس کامل\t", nonzero_count_A_full)


 PCA تعداد مؤلفه‌های غیر صفر ماتریس 	 32619186
تعداد مؤلفه‌های غیر صفر ماتریس کامل	 10279072


In [23]:
## بررسی  پراکندگی ماتریس

is_sparse = sparse.issparse(T_Matrix_PCA)
is_sparse

False

In [24]:
T_Matrix_PCA.format

AttributeError: 'numpy.ndarray' object has no attribute 'format'